# Example subscribing to Kafka topic and receiving messages
<font color='steelblue'>
<br>
<span style="font-family:Arial; font-size:1.6em;">
Receive kafka messages and print it to the console<br>
Use the Producer-Consumer Java code for generating kafka traffic<br>
<br>
Steps to test: <br>
<ol>
    <li> Start the zookeeper, kafka brokers</li>
    <li> Create topic</li>
    <li> Run all the cells</li>
    <li> Run the producer code to generate the messages</li>
    <li> Observe the messages in the console (only 20 will be printed)</li>
</ol>
 </span>

</font>
<br>
<font color='gray'>
<span style="font-family:Arial; font-size:1.4em;">
    <b>Following are the steps in the processing:<br></b>
    <ul>
        <li><b>Read Stream:</b> Create kafka read stream</li>
        <li><b>Extract:</b> Extract the message from the stream</li>
        <li><b>Write Stream:</b> Write the stream to console</li>
    </ul>
</span>
</font>

In [ ]:
# Set up the environment for using pyspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession\
        .builder\
        .appName("Kafka-Spark integration")\
        .getOrCreate()

In [ ]:
 spark.sparkContext.setLogLevel("ERROR")

In [ ]:
# define the topic name
#topicName = "producer-consumer-topic"
topicName = "sales-data"

# define where to start reading from
startFrom = "earliest"

## Read Kafka Stream

In [ ]:
df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "localhost:9092") \
      .option("subscribe", topicName) \
      .option("startingOffsets", startFrom) \
      .load()

In [ ]:
# check if dataframe is streaming?
df.isStreaming

In [ ]:
df.printSchema()

## Extract the messages

In [ ]:
# select the value column
sdf = df.select(col("value").cast(StringType()))

In [ ]:
sdf.printSchema()

## Write the messages to console

In [ ]:
# write the stream to the console
query = sdf.writeStream\
           .outputMode('update')\
           .format('console')\
           .option("truncate", "false")\
           .queryName("kafka-stream")\
           .start()

In [ ]:
# Check active streams
for s in spark.streams.active:
    print("ID:{} | NAME:{}".format(s.id, s.name))

In [ ]:
query.awaitTermination()